### Import related libraries

In [1]:
import pandas as pd
from sqlalchemy import create_engine
from tkinter import simpledialog
import uuid
import numpy as np
import os
import shutil
import pyodbc

### Get Data from the summary table

#### Connect to the Elmod Database

In [3]:
user = 'postgres'
password = simpledialog.askstring(title = 'Enter your password', prompt= 'What is the database password?')
database = 'elmod_db'
host = 'localhost'
port = '5432'
batch_no = simpledialog.askstring (title = "Enter Batch No", prompt = "What is the batch no you want to work on?")
engine = create_engine (f'postgresql://{user}:{password}@{host}:{port}/{database}')
with engine.connect () as conn:
    df = pd.read_sql (f"""
    select *
    FROM summary_table
    WHERE batch_no ='{batch_no}'
    """, conn)
    
    df_stations = pd.read_sql ("""
    SELECT * FROM stations
    """, conn)
    
    df_drops = pd.read_sql ("""
    SELECT A.* FROM drops AS A
    LEFT OUTER JOIN stations AS B
    ON A."GUID" = B."GUID"
    AND A."StationID" = B."StationID"
    WHERE B."Latitude" IS NOT NULL 
    """, conn)

    df_sessions = pd.read_sql ("""
     SELECT * FROM sessions 
    """, conn)
conn.close ()

# In this step do the following
- Correct the data types of the retrieved data
- Find the min and max kp for each file 
- Calculate and report the number of drops for each file
- If there are suspicious drops check them and correct them in the database (postgres)
- Sort the dataframe ascendingly according to the kp
- Check the number of drops for file and decide if they need manual adjustements

In [5]:
df ['road_no'] = pd.to_numeric (df['road_no'])
df ['lane_no'] = pd.to_numeric (df['lane_no'])
df ['start_kp'] = pd.to_numeric (df['start_kp'])
df ['end_kp'] = pd.to_numeric (df['end_kp'])
df ['batch_no'] = pd.to_numeric (df['batch_no'])
df['min_kp'] = None
df['max_kp'] = None
df['num_drops'] = None
list_suspicious_drops = []
for index in df.index:
    df.at[index, 'min_kp'] = min(df.loc[index, 'start_kp'], df.loc[index, 'end_kp'])
    df.at[index, 'max_kp'] = max(df.loc[index, 'start_kp'], df.loc[index, 'end_kp'])
    # df.at[index, 'num_drops'] = df_drops[df_drops['GUID']== df.loc[index, 'guid']]['DropID'].iloc[-1]/ df_drops [df_drops['GUID']== df.loc[index, 'guid']]['StationID'].iloc[-1]
    df.at[index, 'num_drops'] = df_drops[df_drops['GUID']== df.loc[index, 'guid']]['DropID'].count()/ df_stations [df_stations['GUID']== df.loc[index, 'guid']]['StationID'].count()
    if df.at[index, 'num_drops'] % 1 != 0:
        list_suspicious_drops.append (df.loc [index, 'file_name'])
        
df = df.sort_values ('min_kp', ascending = True)

#Report the files to check
print (list_suspicious_drops)

#Report the number of files for each number of drops 
df_num_drops = df[['num_drops', 'file_name']].groupby ('num_drops').count ()
df_num_drops

[]


,file_name
num_drops,
2.0,236


### Group the data in a table that will be used to get the guid for each group and to name the final files later

In [7]:
#Group the data and get the min and max kps as well as the lane type
df_grouped = df.groupby (['road_no', 'dir_cd', 'num_drops']).agg ({'min_kp':'min', 'max_kp':'max', 'lane_no':'max'}).reset_index ()
df_grouped
# df_2136 = df_grouped [df_grouped['road_no']==2136]
df_9051 = df_grouped [df_grouped['road_no']==9051]


In [9]:
df_9051

,road_no,dir_cd,num_drops,min_kp,max_kp,lane_no
114,9051,M,2.0,0,13,2
115,9051,R,2.0,0,13,2


In [11]:
df_get_guid = df
survey_combined = {}
df_surveys = df_9051
for index in df_surveys.index:
    # print (f"{road_no}-{dir_cd}-{lane_no}-{min_kp}-{max_kp}")  ##############################
    road_no = df_surveys.loc [index, 'road_no']
    dir_cd = df_surveys.loc [index, 'dir_cd']
    lane_no = df_surveys.loc [index, 'lane_no']
    min_kp = df_surveys.loc [index, 'min_kp']
    max_kp = df_surveys.loc [index, 'max_kp']
    no_drops = df_surveys.loc [index, 'num_drops']
    # print (no_drops) ##############################
    #Create a filename
    if dir_cd == 'M':
        file_name = f"H{road_no}-{dir_cd}-L{lane_no}-{min_kp}-{max_kp}"
    else:
        file_name = f"H{road_no}-{dir_cd}-L{lane_no}-{max_kp}-{min_kp}"
    
    #Get a list of guid, to be retrieved from drops and stations
    guid_list = df_get_guid[(df_get_guid['road_no']==road_no) & (df_get_guid['dir_cd']==dir_cd) & (df_get_guid['num_drops']==no_drops) ]['guid'].tolist()
    
    list_for_sta = []
    list_for_drops = []
    list_for_sessions = []
    
    #Reverse the list of the guid in case of Reverse surveys
    if dir_cd == 'R':
       guid_list.reverse ()

    for guid in guid_list:
        #Get the dataframe according to the guid
        df_part_sta= df_stations[df_stations['GUID']==guid].sort_values (by = 'StationID')
        df_part_drop= df_drops[df_drops['GUID']==guid].sort_values (by = 'DropID')
        df_part_session = df_sessions [ df_sessions ['GUID'] == guid]
        #Append it to the list 
        list_for_sta.append(df_part_sta)
        list_for_drops.append(df_part_drop)
        list_for_sessions.append (df_part_session)

    #Concat the list 
    df_sta_comb = pd.concat (list_for_sta)
    df_drops_comb = pd.concat (list_for_drops)
    df_sessions_comb = pd.concat (list_for_sessions)
    
    #Create the sequences of the drops and stations and sessions
    #1. For the df_drops_comb
    drop_seq = np.arange (1, len(df_drops_comb)+1)
    df_drops_comb['DropID'] = drop_seq
    station_seq = np.repeat (np.arange (1, int((len(df_drops_comb)/no_drops)+1)), no_drops)
    df_drops_comb['StationID'] = station_seq

    #2. For the df_sta_comb
    station_seq_2 = np.arange (1, int((len(df_drops_comb)/no_drops)+1))
    df_sta_comb ['StationID'] = station_seq_2
    df_sta_comb ['SessionID'] = 1

    #3. For the df_sessions_new
    df_sessions_new = df_sessions_comb.iloc[[0]]
    df_sessions_new.loc[:, 'SessionID'] = 1

    #Drop the GUID column from drops and stations table
    df_drops_comb.drop (columns = ['GUID'], inplace = True)
    df_sta_comb.drop (columns = ['GUID'], inplace = True)

    survey_combined [file_name] = {"Sessions": df_sessions_new,
                                   "Stations":df_sta_comb,
                                   "Drops": df_drops_comb}

In [144]:
# df = survey_combined['H66-M-L2-13-693']['Stations']

In [145]:
# df[df['Latitude']== 24.86736079200095].values

# Upload data to the EMPTY ACCESS FILE

In [13]:
#Enter folder location of empty database:
mdb_file_path = r"C:\Users\Khaled Ahmed\Desktop\fwd_analysis_ASH\Empty_Data_Base.mdb"
location_save_mdb = r"C:\Users\Khaled Ahmed\Desktop\fwd_analysis_ASH\004_Analysis_ASH"


for file, table in survey_combined.items():
    file_name = file + '.mdb'
    file_new_path = os.path.join (location_save_mdb, file_name)
    shutil.copy (mdb_file_path, file_new_path)
    
    #Connect to the Microsoft Access Database
    conn_str = r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'r'DBQ='+file_new_path+';'
                                  
    # Connect to the Access database
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    # Define the Access table name
    for access_table, df in survey_combined [file].items ():
        df= df.reset_index (drop = True)
    
        # Create insert query template with column names enclosed in square brackets
        columns = ", ".join([f"[{col}]" for col in df.columns])
        placeholders = ", ".join(["?" for _ in df.columns])
        insert_query = f"INSERT INTO {access_table} ({columns}) VALUES ({placeholders})"
        
        # Print insert query for debugging
        print("Insert Query:", insert_query)
        
        # Insert DataFrame into Access table
        for index, row in df.iterrows():
            print("Row:", tuple(row))  # Print the row for debugging
            cursor.execute(insert_query, tuple(row))
        
        # Commit the transaction
        conn.commit()
        
    # Close the connection
    cursor.close()
    conn.close()

Insert Query: INSERT INTO Sessions ([SessionID], [GUID], [Program], [Operator], [Driver], [Trailer], [Electronics], [TestSetup], [Units], [Date], [FacilityName], [FacilityCode], [FacilityDirection], [FacilityType], [FacilityLanes], [SectionName], [SectionCode], [SectionDirection], [District], [Class], [Start], [End], [Lane], [LaneNumber], [Traffic], [Smoothing], [StationMin], [StationMax], [Radius], [X1], [X2], [X3], [X4], [X5], [X6], [X7], [X8], [X9], [X10], [X11], [X12], [X13], [X14], [X15], [Y1], [Y2], [Y3], [Y4], [Y5], [Y6], [Y7], [Y8], [Y9], [Y10], [Y11], [Y12], [Y13], [Y14], [Y15]) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
Row: (1, '7CCCBB3BBFE14D688235B004868022C', '2.9.3', '', '', '8012-058', 'CP15-959', 'MOT TEST 50KN 4 Drops Standard', 'Metric', Timestamp('2024-01-16 11:00:54'), 'MOT', '', 0, '', 0, 'EIN HAWAS ROAD', '', 0, '', '', '0'

In [18]:
cursor.close()
conn.close()

ProgrammingError: Attempt to use a closed cursor.